### Build a Arc-I Model

<img src="https://raw.githubusercontent.com/NTMC-Community/MatchZoo/2.0/artworks/matchzoo-logo.png" alt="logo" style="width:600px;float: center"/>

This is a tutorial on training *Arc-I Model* [Hu et al. 2014](http://papers.nips.cc/paper/5550-convolutional-neural-network-architectures-for-matching-natural-language-sentences.pdf) model with [MatchZoo](https://github.com/faneshion/MatchZoo)for **classification task**. We use [QuoraQP](https://data.quora.com/First-Quora-Dataset-Release-Question-Pairs) as the example benchmark data set to show the usage.

*To walk through this notebook, you need approx 90 minutes.*

-------

In [1]:
import sys
print(sys.path)

['', '/home/pangliang/nips/venv/lib/python36.zip', '/home/pangliang/nips/venv/lib/python3.6', '/home/pangliang/nips/venv/lib/python3.6/lib-dynload', '/usr/local/python3/lib/python3.6', '/home/pangliang/nips/venv/lib/python3.6/site-packages', '/home/pangliang/nips/playground_pl', '/home/pangliang/matching/MatchZoo_New', '/home/pangliang/nips/venv/lib/python3.6/site-packages/IPython/extensions', '/home/pangliang/.ipython']


**TL;DR**

The following code block illustrates the main workflow of how to train a Arc-I model. 

```python
from matchzoo import preprocessor
from matchzoo import generators
from matchzoo import models

train, test = ... # prepare your training data and test data.

arci_preprocessor = preprocessor.ArcIPreprocessor()
processed_tr = arci_preprocessor.fit_transform(train, stage='train')
processed_te = arci_preprocessor.fit_transform(test, stage='predict')


generator_tr = generators.PointGenerator(processed_tr)
generator_te = generators.PointGenerator(processed_te)
# Example, train with generator, test with the first batch.
X_te, y_te = generator_te[0]

arci_model = models.ArcIModel()
arci_model.guess_and_fill_missing_params()
arci_model.build()
arci_model.compile()
arci_model.fit_generator(generator_tr)
# Make predictions
predictions = arci_model.predict([X_te.text_left, X_te.text_right])
```

-----

MatchZoo expect a list of *Quintuple* as training data:

```python
train = [('qid0', 'did0', 'query 0', 'document 0', 'label 0'),
         ('qid0', 'did1', 'query 0', 'document 1', 'label 1'),
          ...,
         ('qid1', 'did2', 'query 1', 'document 2', 'label 3')]
```

The corresponded columns are `(text_left_id, text_right_id, text_left, text_right, label)`. For Information Retrieval task, *text_left* is referred as *query*, and *text_right* is document.

For the test case, MatchZoo expect a list of *Quadruple* (we do not need labels) as input:

```python
test = [('qid9', 'did5', 'query 9', 'document 5'),
         ...,
        ('qid2', 'did7', 'query 2', 'document 7')]
```

### Table of Content

+ Prepare QuoraQP dataset
    - Download
    - Load
    - Adjustment
+ Preprocessing
+ Data Generator
+ Model Training
    - Initialize
    - Hyper-Parameters
    - Make Prediction
    - Model Persistence
- Reference

### Prepare Quora Question Pair dataset

#### Download

We take QuoraQP as the example benchmark dataset to show the usage of MatchZoo. Firstly you need to downlowd the data and uncompress the data, currently you need to download the dataset via [kaggle](https://www.kaggle.com/c/quora-question-pairs/data). Unzip the data, you'll get

- train.csv
- test.csv

#### Load & Adjustment

The *train/test* files of QuoraQP are *train.csv*,  *test.csv* under the uncompressed folder QuoraQP. 

We can convert this format to the expected input format of MatchZoo.

In [6]:
data_folder = '../../data/quoraqp/'

import pandas as pd
import numpy as np

def read_data(file_path, stage):
    df = pd.read_csv(file_path, dtype={'question1': 'str', 'question2': 'str'}, keep_default_na=False)
    print(df.describe())
    if stage == 'train':
        df = df[['qid1', 'qid2', 'question1', 'question2', 'is_duplicate']]
    elif stage == 'predict':
        df = df[['question1', 'question2']]
        # assign ids to left and right
        q_a = pd.unique(df.values.ravel())
        # add index for each
        q_a = dict(map(lambda t: (t[1], t[0]), enumerate(q_a)))
        # assign id
        df['qid1'] = df['question1'].map(q_a)
        df['qid2'] = df['question2'].map(q_a)
        # change the order of columns
        cols = ['qid1', 'qid2', 'question1', 'question2']
        df = df[cols]
    # convert dataframe into list of tuples
    qa_pairs = [tuple(x) for x in df.values]
    return qa_pairs
    

train_all = read_data(data_folder + 'train.csv', stage='train')

train = []
test = []
for i in range(len(train_all)):
    if i % 5 == 0:
        test.append(tuple(train_all[i][:-1]))
    else:
        train.append(train_all[i])

# if you need test on unseen test set use this code
# test_all = read_data(data_folder + 'test.csv', stage='predict')


                  id           qid1           qid2   is_duplicate
count  404290.000000  404290.000000  404290.000000  404290.000000
mean   202144.500000  217243.942418  220955.655337       0.369198
std    116708.614503  157751.700002  159903.182629       0.482588
min         0.000000       1.000000       2.000000       0.000000
25%    101072.250000   74437.500000   74727.000000       0.000000
50%    202144.500000  192182.000000  197052.000000       0.000000
75%    303216.750000  346573.500000  354692.500000       1.000000
max    404289.000000  537932.000000  537933.000000       1.000000


### Preprocessing


#### Download

We first download the GLoVe pre-trained word embeddings. Be pationt - this takes a long time related to your network service. Alternatively, you can download it via another downloading tool and put **glove.840B.300d.txt** in the directory **../../data/embedding**.

In [ ]:
!mkdir -p ../../data/embedding
!wget -O ../../data/embedding/glove.840B.300d.zip http://nlp.stanford.edu/data/glove.840B.300d.zip 
!unzip -o ../../data/embedding/glove.840B.300d.zip -d ../../data/embedding

#### Preproceesing with pre-trained embeddings

We run the processor in three lines of code.  
If we use **the randomly initialized embeddings**, just keep default value for *embedding_file*, and the preprocessing code is as follows:
```python
from matchzoo import preprocessor
arci_preprocessor = preprocessor.ArcIPreprocessor()
processed_tr = arci_preprocessor.fit_transform(train, stage='train')
processed_te = arci_preprocessor.fit_transform(test, stage='predict')
```

In [8]:
# Initialize a arci preprocessor.
from matchzoo import preprocessor
arci_preprocessor = preprocessor.ArcIPreprocessor(embedding_file='../../data/embedding/glove.840B.300d.txt')
processed_tr = arci_preprocessor.fit_transform(train, stage='train')
processed_te = arci_preprocessor.fit_transform(test, stage='predict')

Start building vocabulary & fitting parameters.
241934it [01:25, 2843.19it/s]
247429it [01:28, 2780.61it/s]
Some words are not shown in term_index(74174). Total number of words are 74175.
2196017it [01:00, 36364.00it/s]
Start processing input data for train stage.
241934it [01:40, 2417.84it/s]
247429it [01:44, 2377.07it/s]
Start processing input data for predict stage.
71733it [00:29, 2434.85it/s]
71840it [00:30, 2334.72it/s]


**What is `processed_tr`?**

`processed_tr` is a **MatchZoo DataPack** data structure (see `matchzoo/datapack.py`). It contains 
1. A *2-columns* `pandas DataFrame` called `left` to host all the pre-processed records including index and processed text to store `text_left` and `id_left`.
2. A *2-columns* `pandas DataFrame` called `right` to host all the pre-processed records including index and processed text to store `text_right` and `id_right`.
3. A *2-columns* `pandas DataFrame` called `relation` to host all the pre-processed records including index and index mapping `id_left` and `id_right`.
4. A `context` property (dictionary) consists of all the parameters fitted during pre-processing. 


The `fit_transform` method is a sequential combination of two methods:

1. Fit parameters using the `fit` function, this only happens when `stage='train'`.
2. Transform data into expected format.

So the previous preprocessing code in test stage can also be written as:

```python
processed_te = arci_preprocessor.transform(test, stage='predict')
```

As described, the fitted parameters were stored in `context` property, to access the context, just call:

```python
print(processed_tr.context)
```
An example:

In [9]:
print('vocab size: ', len(processed_tr.context['term_index']))

vocab size:  74174


**What has been stored in the `context?`** 

We stored `term_index` and `input_shapes` in the context property. 


**What is `arci_preprocessor` actually doing?**

The `arci_preprocessor` is calling a sequence of `process_units`. Each `process_unit` is designed to perform one atom operation on input data. For instance, in `arci_preprocessor`, we called:


1. TokenizeUnit: Perform tokenization on raw input data.
2. LowercaseUnit: Transform all tokens into lower case.
3. PuncRemovalUnit: Remove all the punctuations.
4. StopRemovalUnit: Remove all the stopwords.

Depending on whether using the pre-trained embeddings, the preprocessor loads the embedding from an embedding file or randomly initializes the embeddings.


----

### Data Generation

For memory efficiency, we expect you to use **generator** to generate batches of data on the fly. For example, we can create a **PointGenerator** as follows:

In [10]:
from matchzoo import generators
from matchzoo import tasks
generator_tr = generators.PointGenerator(inputs=processed_tr, task=tasks.Classification(), batch_size=64, stage='train')
generator_te = generators.PointGenerator(inputs=processed_te, task=tasks.Classification(), batch_size=64, stage='predict')

To get the first batch of trainig data, just call `X_train, y_train = generator[0]`.

**What is PointGenerator?**
**PointGenerator** is this case, it is assumed that each query-document pair in the training data has a numerical or ordinal score. Then the problem can be approximated by a regression/Classification problem — given a single query-document pair, predict its score.

A number of existing supervised machine learning algorithms can be readily used for this purpose. Ordinal regression and classification algorithms can also be used in pointwise approach when they are used to predict the score of a single query-document pair, and it takes a small, finite number of values.

**What is PairGenerator?**
In this case, the problem is approximated by a classification problem — learning a binary classifier that can tell which document is better in a given pair of documents.

In MatchZoo, **PairGenerator** generate one positive & `num_neg` negative examples per pair. As an example, to train an Arc-I model (for document ranking), we use `num_neg=4`. 

**What is ListGenerator?**
This generator try to directly optimize the value of evaluation measures, averaged over all queries in the training data. 

Chosse the appropriate generator based on your `task`.

----

### Train Your Arc-I Model

To train a Arc-I model, we need to create an instance of ArcIModel:

In [11]:
from matchzoo import models
arci_model = models.ArcIModel()

Then, we need to set hyper-parameters to our Arc-I Model. In general, there are **two types of hyper-parameters**:

**Required parameters**: For Arc-I, if you are using the pre-trained embeddings, you're required to set the embedding_mat and the size manually!

In [12]:
# The fitted parameters is stored in the `context` property of pre-processor instance during the training stage.
from matchzoo import losses
from matchzoo import tasks
arci_model.params['task'] = tasks.Classification()
arci_model.embedding_mat = processed_tr.context['embedding_mat']

**Tunable parameters**: For Arc-I, you're allowed to tune these parameters:

```python
from matchzoo import tasks

        
params = {
            'input_shapes': [(32,), (32,)] # Lengths of matching texts.
            'optimizer': 'adam' # By default, we use sgd. See keras optimizer.
            'trainable_embedding': False # Whether finetune the embeddings.
            'num_blocks': 1 # Number of the convolution layers.
            'left_kernel_count': [32] # Number of filters in convolution. See keras Conv1D.
            'left_kernel_size': [3] # The length of the 1D convolution window. See keras Conv1D.
            'right_kernel_count': [32] # Number of filters in convolution. See keras Conv1D.
            'right_kernel_size': [3] # The length of the 1D convolution window. See keras Conv1D.
            'activation': 'relu' # Activation function in convolution. See keras Conv1D.
            'left_pool_size': [2] # Size of the max pooling windows for the left text. See keras MaxPooling1D.
            'right_pool_size': [2] # Size of the max pooling windows for rhe right text. See keras MaxPooling1D.
            'padding': 'same' # Padding mode. See keras padding in Conv1D.   
            'dropout_rate': 0.0 # Probability of an element to be zeroed. See keras Dropout.
            'embedding_random_scale': 0.2 # The range of the random initialized embedding. 
            'task': tasks.Classification, # Default Classification, you can use tasks.Ranking
            'loss': 'categorical_crossentropy', # categorical_crossentropy, see keras loss.
            'metric': 'acc', # Accuracy by default, see keras metric.
         }
```

In [13]:
arci_model.guess_and_fill_missing_params()
print('arci parameters: ', arci_model.params)

arci parameters:  name                          ArcIModel
model_class                   <class 'matchzoo.models.arci_model.ArcIModel'>
input_shapes                  [(32,), (32,)]
task                          <matchzoo.tasks.classification.Classification object at 0x7fc65ef914a8>
metrics                       ['acc']
loss                          categorical_crossentropy
optimizer                     adam
trainable_embedding           False
embedding_dim                 300
vocab_size                    74175
num_blocks                    1
left_kernel_count             [32]
left_kernel_size              [3]
right_kernel_count            [32]
right_kernel_size             [3]
activation                    relu
left_pool_size                [2]
right_pool_size               [2]
padding                       same
dropout_rate                  0.0
embedding_mat                 [[ 0.        0.        0.       ...  0.        0.        0.      ]
 [-0.059527  0.13702  -1.1131   ...  0.74374 

#### Model Training

To train the model after all the parameters were settled, call:

In [14]:
arci_model.build()
arci_model.compile()
# Fit the arci model on generator.
arci_model.fit_generator(generator_tr, steps_per_epoch=200, epochs=10)
# Make predictions on the first batch of test data
X_te, y_te = generator_te[0]
predictions = arci_model.predict([X_te.text_left, X_te.text_right])

Epoch 1/10
200/200 [==============================] - 6s 31ms/step - loss: 0.6037 - acc: 0.6766
Epoch 2/10
200/200 [==============================] - 5s 26ms/step - loss: 0.5728 - acc: 0.7068
Epoch 3/10
200/200 [==============================] - 5s 25ms/step - loss: 0.5650 - acc: 0.7104
Epoch 4/10
200/200 [==============================] - 5s 26ms/step - loss: 0.5535 - acc: 0.7182
Epoch 5/10
200/200 [==============================] - 5s 26ms/step - loss: 0.5421 - acc: 0.7270
Epoch 6/10
200/200 [==============================] - 5s 25ms/step - loss: 0.5471 - acc: 0.7280
Epoch 7/10
200/200 [==============================] - 5s 26ms/step - loss: 0.5354 - acc: 0.7334
Epoch 8/10
200/200 [==============================] - 5s 25ms/step - loss: 0.5343 - acc: 0.7357: 1s - 
Epoch 9/10
200/200 [==============================] - 5s 26ms/step - loss: 0.5388 - acc: 0.7305
Epoch 10/10
200/200 [==============================] - 5s 25ms/step - loss: 0.5364 - acc: 0.7337


#### Make Prediction

In [15]:
for id_left, id_right, pred in zip(X_te.id_left, X_te.id_right, predictions):
    print("{}/{} is predicted as {}".format(id_left, id_right, pred))

33028/43757 is predicted as [0.4979838 0.5020162]
115696/378134 is predicted as [0.8761945  0.12380555]
205305/205306 is predicted as [0.9504754  0.04952463]
49383/162039 is predicted as [0.5655674 0.4344326]
480633/480634 is predicted as [0.67904896 0.3209511 ]
416931/416932 is predicted as [0.45275682 0.54724324]
387313/387314 is predicted as [0.9402801  0.05971995]
388271/388272 is predicted as [0.7440659  0.25593412]
248810/248811 is predicted as [0.6187144 0.3812856]
38511/38512 is predicted as [0.33405593 0.66594404]
37662/37663 is predicted as [0.4023736 0.5976264]
246314/246315 is predicted as [0.59830076 0.40169924]
109229/40719 is predicted as [0.32115906 0.6788409 ]
433070/433071 is predicted as [0.7403322  0.25966778]
132063/132064 is predicted as [0.8728965  0.12710348]
309263/309264 is predicted as [0.8540141  0.14598592]
65248/134425 is predicted as [0.11496417 0.88503575]
357101/357102 is predicted as [0.914261   0.08573897]
273353/126678 is predicted as [0.3753102  0.6

#### Model Persistence

You can persist your trained model using `model.save()` and `load_model` function:

```python
from matchzoo import engine
# Save the model to dir.
arci_model.save('/your-model-saved-path')
# And load the model from dir.
engine.load_model('/your-model-saved-path')
```

## Reference

[Hu et al. 2014] Baotian Hu, Zhengdong Lu, Hang Li, and Qingcai Chen. "Convolutional neural network architectures for matching natural language sentences." In Advances in neural information processing systems (NIPS), pp. 2042-2050. 2014.